1. Import libraries

In [ ]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

1a. Set CLIENT_ID and CLIENT_SECRET

In [ ]:
CLIENT_ID = "c238a688346d455ab863d901b28a3ffe"
CLIENT_SECRET = "27783430f4f346db8a3dbdd666fd90af"

client_credentials_manager = SpotifyClientCredentials(client_id = CLIENT_ID, client_secret = CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

2. Set URL

In [ ]:
url =  'https://billboard.com/charts/billboard-200'

3. Make a request to the server

In [ ]:
page = requests.get(url)
page.status_code

4. Create a list to store song titles

In [ ]:
song_titles = []

In [ ]:
if page.status_code == 200:
  soup = BeautifulSoup(page.content, 'html.parser')

  # 5. Find all the ul tags with 'o-chart-result-list-row' class
  ul_tags = soup.find_all('ul', class_ = 'o-chart-results-list-row')
  # print(ul_tags)

  for ul_tag in ul_tags:
    # 6. find the first h3 tag
    h3 = ul_tag.find('h3')
    song_titles.append(h3.get_text().strip())

  # 7. Prepare an empty list to store track data
  track_data = []

  # 8. Use Spotipy to search for song and get the audio features
  for title in song_titles:

    # 9. Search for the song 
    results = sp.search(q = title, limit = 1, type = 'track')

    # to ensure that ['tracks']['items'] exists
    if results['tracks']['items']:
      track = results['tracks']['items'][0]
      track_id = track['id']

      # 10. Get the track features
      track_features = sp.audio_features(track_id)[0]
      # print(track_features)
        
      # 11. Create a dictionary to store track info
      track_info = {
        'artist': track['artists'][0]['name'],
        'track_name': track['name'],
        'danceability': track_features['danceability'],
        'liveness': track_features['liveness'],
        'loudness': track_features['loudness']
      }

      track_data.append(track_info)
    
    # print(track_data) # will take a long time to run

In [ ]:
df = pd.DataFrame(track_data)
df

Isolate dataframe to certain columns

In [ ]:
features = df[['danceability', 'liveness', 'loudness']]
features

Get correlation of features (danceability, liveness, loudness)

In [ ]:
features.corr()

Create heatmap

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.heatmap(features.corr(), annot = True, fmt = '.2f', cmap = 'coolwarm')
plt.title('Correlation Heatmap of Audio Features')
plt.show()  

Build scatter plot for correlation between danceability and loudness

In [ ]:
features.plot(kind ='scatter', x = 'danceability', y = 'loudness')
plt.title('Danceability vs Loudness')
plt.show()

Build scatter plot for correlation between danceability and liveness

In [ ]:
features.plot(kind ='scatter', x = 'danceability', y = 'liveness')
plt.title('Danceability vs Liveness')
plt.show()